In [1]:
import os
import pandas as pd

HORNSGATAN_HOME = os.environ["HORNSGATAN_HOME"]
os.chdir(HORNSGATAN_HOME)
print("Current directory:", os.getcwd())

from src.tools import mytools


date = "2020-01-02"
detector = "e2w_in"
path  = "data/daily_splitted_data/"


Current directory: /home/kaveh/projects/Hornsgatan


In [ ]:
data = pd.read_csv(f"{path}data_{date}.csv")
data.head()

,detector_id,time_detector_real,speed_detector_real,day,date
0,e2w_in,1577923226,48,Thursday,2020-01-02
1,e2w_in,1577923230,43,Thursday,2020-01-02
2,w2e_in,1577923232,31,Thursday,2020-01-02
3,e2w_in,1577923233,32,Thursday,2020-01-02
4,e2w_in,1577923235,32,Thursday,2020-01-02


In [3]:
data = data[data["detector_id"]==detector]
data.head()

,detector_id,time_detector_real,speed_detector_real,day,date
0,e2w_in,1577923226,48,Thursday,2020-01-02
1,e2w_in,1577923230,43,Thursday,2020-01-02
3,e2w_in,1577923233,32,Thursday,2020-01-02
4,e2w_in,1577923235,32,Thursday,2020-01-02
5,e2w_in,1577923237,31,Thursday,2020-01-02


In [4]:
len(data)

9481

In [5]:
from src.tools import mytools


In [6]:
#mytools.delete_all_except_list(f"data/sim_intermediate_data/", [".gitkeep"])